In [ ]:
pip install torch transformers

In [ ]:
pip install datasets

### Formality data ==> Bi-directional Formality data

In [ ]:
import pandas as pd
from datasets import Dataset

# Load dataset (Ensure CSV has 'informal' and 'formal' columns)
df = pd.read_csv("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/Formality_data.csv").reset_index(drop=True)

# Verify dataset columns
assert 'informal' in df.columns and 'formal' in df.columns, "Dataset must contain 'informal' and 'formal' columns"

# Remove any rows with missing values
df = df.dropna(subset=['informal', 'formal'])

# Prepare bidirectional samples
def create_bidirectional_dataset(df):
    """Generates bidirectional pairs from the original dataset."""
    bidirectional_data = []

    for informal, formal in zip(df["informal"], df["formal"]):
        # Informal → Formal
        bidirectional_data.append({
            "input": "informal to formal: " + informal,
            "target": formal
        })

        # Formal → Informal
        bidirectional_data.append({
            "input": "formal to informal: " + formal,
            "target": informal
        })

    return Dataset.from_list(bidirectional_data)

# Generate the bidirectional dataset
bidirectional_dataset = create_bidirectional_dataset(df)

# Save the bidirectional dataset for training
bidirectional_dataset.to_csv("/content/drive/MyDrive/TST/bidirectional_dataset.csv", index=False)

# mT5 model training

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from torch.optim import AdamW
from datasets import load_dataset


# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load preprocessed bidirectional dataset (Ensure CSV has 'input' and 'target' columns)
dataset_path = '/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/bidirectional_dataset.csv'
dataset = load_dataset('csv', data_files={'train': dataset_path})

# Verify dataset structure
print(dataset)

# Load mT5 Tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

# Preprocessing function for tokenization
def preprocess_function(examples):
    """Tokenize inputs and targets with padding/truncation."""
    if 'input' not in examples or 'target' not in examples:
        raise ValueError("Dataset must contain 'input' and 'target' columns.")

    model_inputs = tokenizer(
        examples["input"], max_length=128, truncation=True, padding="max_length"
    )

    labels = tokenizer(
        examples["target"], max_length=128, truncation=True, padding="max_length"
    ).input_ids

    # Replace padding tokens with -100 for loss masking
    labels = [[-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels]

    model_inputs["labels"] = labels
    return model_inputs

# Apply preprocessing
tokenized_dataset = dataset['train'].map(preprocess_function, batched=True)

# Split into train and eval sets
dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Load Model and Move to Device
model = T5ForConditionalGeneration.from_pretrained("google/mt5-small").to(device)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Increased number of epochs
    learning_rate=3e-5,  # Adjusted learning rate
    logging_dir="./logs",
    save_total_limit=2,
)

# Advanced Optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)

# Trainer Setup
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, None),  # Custom optimizer
)

# Train the Model
print("\n🚀 Starting fine-tuning...")
trainer.train()

# Save the Final Model
model.save_pretrained("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/mt5_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/mt5_model")

print("\n Model saved successfully!")

Using device: cuda
DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 47036
    })
})
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly

In [ ]:
# Inference Functions for Both Directions
def generate_sentence(input_text, direction="informal_to_formal"):
    """
    Generates a sentence in the specified direction:
    - "informal_to_formal" → Converts informal to formal
    - "formal_to_informal" → Converts formal to informal
    """
    model.eval()

    prompt = f"{'informal to formal' if direction == 'informal_to_formal' else 'formal to informal'}: {input_text}"

    # Tokenize and move to device
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # Generate
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

  # Example Testing
print("\n🚀 Testing the Model")
print("Informal → Formal:", generate_sentence("तू कहाँ जा रहा है?", direction="informal_to_formal"))
print("Formal → Informal:", generate_sentence("आप कहाँ जा रहे हैं?", direction="formal_to_informal"))

🚀 Testing the Model
Informal → Formal: आप कहाँ जा रहे हैं?
Formal → Informal: तुम कहाँ जा रहे हो?


## mT5 model inference

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Replace with your model's path or checkpoint name
model_path = "/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/mt5_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Batch inference for multiple sentences
def batch_generate(sentences, direction="informal_to_formal", max_length=128, num_beams=5):
    """
    Generates sentences in batch:
    - direction: "informal_to_formal" or "formal_to_informal"
    - max_length: maximum length of the output sentences
    - num_beams: number of beams for beam search
    """
    model.eval()

    # Add direction prefix to each sentence
    prompts = [
        f"{'informal to formal' if direction == 'informal_to_formal' else 'formal to informal'}: {sentence}"
        for sentence in sentences
    ]

    # Tokenize the batch
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)

    # Generate predictions
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True
        )

    # Decode predictions
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Example testing with multiple sentences
informal_sentences = [
    "खाना खा।",
    "बैठ जा।",
    "जल्दी कर।",
    "खाना मस्त था, थैंक्स!",
    "इस होटल की सर्विस बढ़िया है।",
    "ऑफिस में आ जा, भाई!",
    "फटाफट पेपर दिखा दे।",
    "नई जॉब के लिए एप्लाई कर रहा क्या?",
    "पेपर की तैयारी हो गई क्या?",
    "तेरी पढ़ाई तो टॉप लेवल की है।",
    "रिजल्ट चेक कर ले।",
    "तेरी शादी में आके बहुत मज़ा आया।",
    "शादी में आएगा ना?",
    "टाइम पे काम कर भाई।",
    "यार मीटिंग में टाइम से आ जाना।",
    "तेरी तबियत ठीक चल रही है।",
    "वो मूवी देखी क्या?",
    "तेरी फेवरेट मूवी कौन सी है?",
    "तेरे घरवालों को दिवाली मुबारक!",
    "स्कूल में सबका स्वागत है।",
    "तेरा स्कूल बहुत अच्छा है।",
    "सिलेबस फॉलो कर ले।",
    "तेरी पढ़ाई सही चल रही है।",
    "पेपर क्लियर किया क्या?",
    "तू उससे प्यार करता है क्या?",
    "तेरी यारी लाजवाब है।",
    "तेरा प्यार सच्चा लगता है।",
]

formal_sentences = [
    "खाना खाइए।",
    "कृपया बैठ जाइए।",
    "कृपया शीघ्र करें।",
    "भोजन अत्यंत स्वादिष्ट था, धन्यवाद।",
    "आपका व्यंजन बहुत लजीज था।",
    "क्या आप भोजन का स्वाद पसंद कर रहे हैं?",
    "कृपया अपने दस्तावेज़ प्रस्तुत करें।",
    "आपकी नौकरी में तरक्की हुई है।",
    "क्या आप नई नौकरी के लिए आवेदन कर रहे हैं?",
    "आपकी शिक्षा अत्यंत प्रशंसनीय है।",
    "क्या आपने अपनी परीक्षा की तैयारी पूरी कर ली?",
    "आपकी शिक्षा का स्तर अत्यंत उच्च है।",
    "कृपया परीक्षा परिणाम देखें।",
    "आपका विवाह समारोह बहुत भव्य था।",
    "क्या आप विवाह में सम्मिलित होंगे?",
    "आपके सहयोग के लिए धन्यवाद।",
    "कृपया समय पर कार्य करें।",
    "आपकी मेहनत अत्यंत सराहनीय है।",
    "कृपया मीटिंग में समय पर पहुंचें।",
    "कृपया नियमित रूप से व्यायाम करें।",
    "क्या आपने वह फिल्म देखी है?",
    "आपकी पसंदीदा फिल्म कौन सी है?",
    "त्योहार की हार्दिक शुभकामनाएँ।",
    "आपके परिवार को दीपावली की बधाई।",
    "क्या आप होली खेलेंगे?",
    "विद्यालय में सभी छात्रों का स्वागत है।",
    "महाविद्यालय का वातावरण शांतिपूर्ण है।",
    "आपकी कॉलेज की पढ़ाई सराहनीय है।",
    "क्या आप परीक्षा में उत्तीर्ण हुए हैं?",
    "क्या आप उससे प्रेम करते हैं?",
    "आपकी दोस्ती सराहनीय है।",
    "आपके प्रेम में सच्चाई झलकती है।",
]

print("=== mt5 model output ===")
# Batch inference
print("\n🚀 Informal → Formal (Batch)")
formal_outputs = batch_generate(informal_sentences, direction="informal_to_formal")
for informal, formal in zip(informal_sentences, formal_outputs):
    print(f"Informal: {informal}")
    print(f"Formal: {formal}\n")

print("\n🚀 Formal → Informal (Batch)")
informal_outputs = batch_generate(formal_sentences, direction="formal_to_informal")
for formal, informal in zip(formal_sentences, informal_outputs):
    print(f"Formal: {formal}")
    print(f"Informal: {informal}\n")


=== mt5 model output ===

🚀 Informal → Formal (Batch)
Informal: खाना खा।
Formal: कृपया खाना खाइए।

Informal: बैठ जा।
Formal: कृपया बैठ जाइए।

Informal: जल्दी कर।
Formal: कृपया जल्दी करें।

Informal: खाना मस्त था, थैंक्स!
Formal: यह खाना बहुत अच्छा था।

Informal: इस होटल की सर्विस बढ़िया है।
Formal: इस होटल की सर्विस बहुत सराहनीय है।

Informal: ऑफिस में आ जा, भाई!
Formal: कृपया ऑफिस में आ जाइए।

Informal: फटाफट पेपर दिखा दे।
Formal: कृपया मुझे फटाफट पेपर दिखा दीजिए।

Informal: नई जॉब के लिए एप्लाई कर रहा क्या?
Formal: क्या आप नई जॉब के लिए एप्लाई कर रहे हैं?

Informal: पेपर की तैयारी हो गई क्या?
Formal: क्या इस पेपर की तैयारी हो गई?

Informal: तेरी पढ़ाई तो टॉप लेवल की है।
Formal: आपकी पढ़ाई तो टॉप लेवल की है।

Informal: रिजल्ट चेक कर ले।
Formal: कृपया रिजल्ट चेक करें।

Informal: तेरी शादी में आके बहुत मज़ा आया।
Formal: आपकी शादी में आके बहुत मज़ा आया।

Informal: शादी में आएगा ना?
Formal: क्या आप शादी में आएंगे?

Informal: टाइम पे काम कर भाई।
Formal: कृपया टाइम पे काम करें।

Informal: य

## mT5 performance evaluation

In [ ]:
pip install evaluate sacrebleu rouge_score bert_score

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
import evaluate

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load trained T5 model and tokenizer
model_path = "/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/mt5_finetuned2_model"  # Update model path
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)

# Load test dataset
test_data = pd.read_csv("/content/drive/MyDrive/TST/test.csv")
# test_data = df[:500]

# Load Evaluation Metrics
bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")
bertscore_metric = evaluate.load("bertscore")

# Load GPT-2 for Perplexity Calculation (Fluency)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Function to generate formal sentences using T5
def generate_formal_sentence(informal_text):
    input_text = "informal to formal: " + informal_text
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    with torch.no_grad():
        output_ids = model.generate(input_ids,
                                    max_length=128,
                                    num_beams=5,
                                    repetition_penalty=2.0)  # Better output quality

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Function to calculate Perplexity (Lower = Better Fluency)
def calculate_perplexity(sentence):
    inputs = gpt2_tokenizer(sentence, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        loss = gpt2_model(inputs, labels=inputs).loss
    return torch.exp(loss).item()

# Evaluate model
correct_predictions = 0
total_samples = len(test_data)
predictions, references, perplexities = [], [], []

for i, row in test_data.iterrows():
    informal = row["informal"]
    formal = row["formal"]

    predicted_formal = generate_formal_sentence(informal)

    predictions.append(predicted_formal)
    references.append([formal])  # BLEU expects a list of references
    perplexities.append(calculate_perplexity(predicted_formal))  # Fluency score

    if predicted_formal.strip() == formal.strip():
        correct_predictions += 1

# Compute evaluation metrics
accuracy = correct_predictions / total_samples * 100
bleu_score = bleu_metric.compute(predictions=predictions, references=references)["score"]
rouge_scores = rouge_metric.compute(predictions=predictions, references=[ref[0] for ref in references])
bertscore_results = bertscore_metric.compute(predictions=predictions, references=[ref[0] for ref in references], lang="hi")
avg_perplexity = sum(perplexities) / len(perplexities)

# Print results
print("\n===== mT5 Evaluation Results on New Format (input → target) =====")
print(f"🔹 Accuracy: {accuracy:.2f}%")
print(f"🔹 BLEU Score: {bleu_score:.2f}")
print(f"🔹 ROUGE Scores: {rouge_scores}")
print(f"🔹 BERTScore Precision: {sum(bertscore_results['precision']) / len(bertscore_results['precision']):.2f}")
print(f"🔹 BERTScore Recall: {sum(bertscore_results['recall']) / len(bertscore_results['recall']):.2f}")
print(f"🔹 BERTScore F1: {sum(bertscore_results['f1']) / len(bertscore_results['f1']):.2f}")
print(f"🔹 Avg Perplexity: {avg_perplexity:.2f}")

Using device: cuda

===== mT5 Evaluation Results (input → target) =====
🔹 Accuracy: 10.05%
🔹 BLEU Score: 23.31
🔹 ROUGE Scores: {'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'rougeLsum': np.float64(0.0)}
🔹 BERTScore Precision: 0.85
🔹 BERTScore Recall: 0.85
🔹 BERTScore F1: 0.85
🔹 Avg Perplexity: 8.25


# IndicBart model training

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
from huggingface_hub import login

# Optional: Authenticate with Hugging Face
login()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load updated dataset or continue with the same one
dataset_path = '/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/bidirectional_dataset.csv'
dataset = load_dataset('csv', data_files={'train': dataset_path})

# Load tokenizer and model from the previously saved checkpoint
checkpoint_path = "/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/IndicBART_model"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path).to(device)

# Tokenization function
def preprocess_function(examples):
    inputs = tokenizer(
        examples["input"], max_length=128, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        examples["target"], max_length=128, truncation=True, padding="max_length"
    ).input_ids

    labels = [[-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels]
    inputs["labels"] = labels
    return inputs

# Tokenize and split dataset
tokenized_dataset = dataset['train'].map(preprocess_function, batched=True)
dataset_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

# Define updated training arguments
training_args = TrainingArguments(
    output_dir="./results_continue",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,  # Add more epochs
    logging_dir="./logs_continue",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Data collator and trainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Continue training
trainer.train()

# Save the updated model after additional fine-tuning
output_path = "/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/IndicBART_finetune"
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)


Using device: cuda
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-20c46716fdef>:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit

In [ ]:
# Inference Functions for Both Directions
def generate_sentence(input_text, direction="informal_to_formal"):
    """
    Generates a sentence in the specified direction:
    - "informal_to_formal" → Converts informal to formal
    - "formal_to_informal" → Converts formal to informal
    """
    model.eval()

    prompt = f"{'informal to formal' if direction == 'informal_to_formal' else 'formal to informal'}: {input_text}"

    # Tokenize and move to device
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # Generate
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

  # Example Testing
print("\n🚀 Testing the Model")
print("Informal → Formal:", generate_sentence("तू कहाँ जा रहा है?", direction="informal_to_formal"))
print("Formal → Informal:", generate_sentence("आप कहाँ जा रहे हैं?", direction="formal_to_informal"))

🚀 Testing the Model
Informal → Formal: आप कहाँ जा रहे हैं?
Formal → Informal: तू कहाँ जा रहा है?


## IndicBART model inference

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the IndicBART model and tokenizer
checkpoint_path = "/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/IndicBART_model"

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Batch inference for multiple sentences (cleaned version)
def batch_generate_indicbart(sentences, direction="informal_to_formal", max_length=128, num_beams=5):
    """
    Generates sentences in batch for IndicBART model:
    Ensures only the first sentence is kept if multiple are generated.
    """
    model.eval()

    # Add direction prefix to each sentence
    prompts = [
        f"{'informal to formal' if direction == 'informal_to_formal' else 'formal to informal'}: {sentence}"
        for sentence in sentences
    ]

    # Tokenize the batch
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)

    # Generate predictions
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True
        )

    # Decode and keep only the first sentence
    decoded_outputs = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        # Split by '।' (Hindi full stop) and take the first non-empty part
        first_sentence = text.split('।')[0].strip()
        if first_sentence:  # Add '।' back if needed
            first_sentence += '।'
        decoded_outputs.append(first_sentence)

    return decoded_outputs


In [ ]:
# Example testing with multiple sentences
informal_sentences = [
    "खाना खा।",
    "बैठ जा।",
    "जल्दी कर।",
    "खाना मस्त था, थैंक्स!",
    "इस होटल की सर्विस बढ़िया है।",
    "ऑफिस में आ जा, भाई!",
    "फटाफट पेपर दिखा दे।",
    "नई जॉब के लिए एप्लाई कर रहा क्या?",
    "तेरी पढ़ाई तो टॉप लेवल की है।",
    "रिजल्ट चेक कर ले।",
    "शादी में आएगा ना?",
    "टाइम पे काम कर भाई।",
    "यार मीटिंग में टाइम से आ जाना।",
    "तेरी तबियत ठीक चल रही है।",
    "वो मूवी देखी क्या?",
    "तेरी फेवरेट मूवी कौन सी है?",
    "तेरे घरवालों को दिवाली मुबारक!",
    "स्कूल में सबका स्वागत है।",
    "तेरा स्कूल बहुत अच्छा है।",
    "तेरी पढ़ाई सही चल रही है।",
    "पेपर क्लियर किया क्या?",
    "तू उससे प्यार करता है क्या?",
    "तेरा प्यार सच्चा लगता है।",
]

formal_sentences = [
    "खाना खाइए।",
    "कृपया बैठ जाइए।",
    "कृपया शीघ्र करें।",
    "भोजन अत्यंत स्वादिष्ट था, धन्यवाद।",
    "क्या आप भोजन का स्वाद पसंद कर रहे हैं?",
    "कृपया अपने दस्तावेज़ प्रस्तुत करें।",
    "आपकी नौकरी में तरक्की हुई है।",
    "क्या आप नई नौकरी के लिए आवेदन कर रहे हैं?",
    "आपकी शिक्षा का स्तर अत्यंत उच्च है।",
    "कृपया परीक्षा परिणाम देखें।",
    "आपका विवाह समारोह बहुत भव्य था।",
    "क्या आप विवाह में सम्मिलित होंगे?",
    "आपके सहयोग के लिए धन्यवाद।",
    "कृपया समय पर कार्य करें।",
    "आपकी मेहनत अत्यंत सराहनीय है।",
    "कृपया मीटिंग में समय पर पहुंचें।",
    "कृपया नियमित रूप से व्यायाम करें।",
    "क्या आपने वह फिल्म देखी है?",
    "आपकी पसंदीदा फिल्म कौन सी है?",
    "त्योहार की हार्दिक शुभकामनाएँ।",
    "आपके परिवार को दीपावली की बधाई।",
    "क्या आप होली खेलेंगे?",
    "आपकी शिक्षा में प्रगति हो रही है।",
    "आपकी कॉलेज की पढ़ाई सराहनीय है।",
    "कॉलेज की पढ़ाई कठिन है।",
    "क्या आप उससे प्रेम करते हैं?",
    "आपकी दोस्ती सराहनीय है।",
    "आपके प्रेम में सच्चाई झलकती है।"
]

print("=== IndicBART model output ===")

# Batch inference
print("\n🚀 Informal → Formal (Batch)")
formal_outputs = batch_generate_indicbart(informal_sentences, direction="informal_to_formal")
for informal, formal in zip(informal_sentences, formal_outputs):
    print(f"Informal: {informal}")
    print(f"Formal: {formal}\n")

print("\n🚀 Formal → Informal (Batch)")
informal_outputs = batch_generate_indicbart(formal_sentences, direction="formal_to_informal")
for formal, informal in zip(formal_sentences, informal_outputs):
    print(f"Formal: {formal}")
    print(f"Informal: {informal}\n")


=== IndicBART model output ===

🚀 Informal → Formal (Batch)
Informal: खाना खा।
Formal: कृपया खाना खाएं।

Informal: बैठ जा।
Formal: कृपया बैठ जाइए।

Informal: जल्दी कर।
Formal: कृपया जल्दी करें।

Informal: खाना मस्त था, थैंक्स!
Formal: यह भोजन बहुत स्वादिष्ट था।

Informal: इस होटल की सर्विस बढ़िया है।
Formal: यह होटल बहुत अच्छा है।

Informal: ऑफिस में आ जा, भाई!
Formal: कृपया ऑफिस में उपस्थित रहें।

Informal: फटाफट पेपर दिखा दे।
Formal: कृपया मुझे पेपर की कॉपी दिखाएं।

Informal: नई जॉब के लिए एप्लाई कर रहा क्या?
Formal: क्या आप इस पद के लिए आवेदन कर रहे हैं?।

Informal: तेरी पढ़ाई तो टॉप लेवल की है।
Formal: आपकी पढ़ाई तो टॉप पर है।

Informal: रिजल्ट चेक कर ले।
Formal: कृपया रिजल्ट चेक करें।

Informal: शादी में आएगा ना?
Formal: क्या आप विवाह समारोह में शामिल होंगे?।

Informal: टाइम पे काम कर भाई।
Formal: कृपया समय पर कार्य करें।

Informal: यार मीटिंग में टाइम से आ जाना।
Formal: कृपया मीटिंग में समय पर उपस्थित रहें।

Informal: तेरी तबियत ठीक चल रही है।
Formal: आपकी तबियत स्थिर है।

Inform

### IndicBART performance evaluation

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import evaluate
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load trained model and tokenizer
model_path = "/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/IndicBART_model"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_auth_token=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, use_auth_token=True).to(device)

# Load test dataset
test_data = pd.read_csv("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/test.csv")
# test_data = df[:500]

# Load Evaluation Metrics
bleu_metric = evaluate.load("sacrebleu")
meteor_metric = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

# Load GPT-2 for Perplexity Calculation (to measure fluency)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Function to generate formal sentences using T5
def generate_formal_sentence(informal_text):
    input_text = "informal to formal: " + informal_text
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    with torch.no_grad():
        output_ids = model.generate(input_ids,
                                    max_length=128,
                                    num_beams=5,
                                    repetition_penalty=2.0)  # Better output quality

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Function to calculate Perplexity (Lower = Better Fluency)
def calculate_perplexity(sentence):
    inputs = gpt2_tokenizer(sentence, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        loss = gpt2_model(inputs, labels=inputs).loss
    return torch.exp(loss).item()

# Evaluate model
correct_predictions = 0
total_samples = len(test_data)
predictions, references, perplexities = [], [], []

for i, row in test_data.iterrows():
    informal = row["informal"]
    formal = row["formal"]

    predicted_formal = generate_formal_sentence(informal)

    predictions.append(predicted_formal)
    references.append([formal])  # BLEU expects a list of references
    perplexities.append(calculate_perplexity(predicted_formal))  # Fluency score

    if predicted_formal.strip() == formal.strip():
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_samples * 100

# Compute automatic evaluation metrics
bleu_score = bleu_metric.compute(predictions=predictions, references=references)["score"]
bertscore_results = bertscore_metric.compute(predictions=predictions, references=references, lang="hi")
avg_perplexity = sum(perplexities) / len(perplexities)  # Average perplexity

# Print Results
print("\n===== IndicBART Model Evaluation Results =====")
print(f"🔹 BLEU Score: {bleu_score:.2f}")
print(f"🔹 BERTScore Precision: {sum(bertscore_results['precision']) / len(bertscore_results['precision']):.2f}")
print(f"🔹 BERTScore Recall: {sum(bertscore_results['recall']) / len(bertscore_results['recall']):.2f}")
print(f"🔹 BERTScore F1: {sum(bertscore_results['f1']) / len(bertscore_results['f1']):.2f}")
print(f"🔹 Perplexity (Lower = Better Fluency): {avg_perplexity:.2f}")

Using device: cuda

===== IndicBART Model Evaluation Results =====
🔹 BLEU Score: 17.84
🔹 BERTScore Precision: 0.81
🔹 BERTScore Recall: 0.82
🔹 BERTScore F1: 0.81
🔹 Perplexity (Lower = Better Fluency): 5.36


# Ensemble Model(mT5+IndicBART)

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load mT5 model and tokenizer
mt5_tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
mt5_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/mt5_model").to(device)
mt5_model.eval()

# Load IndicBART model and tokenizer
indic_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indicbart", use_fast=False)
indic_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/IndicBART_model").to(device)
indic_model.eval()

# Ensemble inference function using prompted format
def ensemble_generate(prompt_text):
    # --- mT5 generation ---
    mt5_inputs = mt5_tokenizer(prompt_text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        mt5_output_ids = mt5_model.generate(
            input_ids=mt5_inputs["input_ids"],
            attention_mask=mt5_inputs["attention_mask"],
            max_length=128,
            num_beams=5,
            early_stopping=True,
        )
    mt5_output = mt5_tokenizer.decode(mt5_output_ids[0], skip_special_tokens=True)

    # --- IndicBART generation ---
    indic_inputs = indic_tokenizer(prompt_text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        indic_output_ids = indic_model.generate(
            input_ids=indic_inputs["input_ids"],
            attention_mask=indic_inputs["attention_mask"],
            max_length=128,
            num_beams=5,
            early_stopping=True,
        )
    indic_output = indic_tokenizer.decode(indic_output_ids[0], skip_special_tokens=True)

    # Return longer output (or use mT5 by default if equal)
    return mt5_output if len(mt5_output) >= len(indic_output) else indic_output

# Wrapper like your previous code
def generate_sentence_ensemble(input_text, direction="informal_to_formal"):
    """
    Uses both mT5 and IndicBART to generate sentence in specified direction.
    """
    prompt = f"{'informal to formal' if direction == 'informal_to_formal' else 'formal to informal'}: {input_text}"
    return ensemble_generate(prompt)

# Test Examples
print("\n🚀 Testing Ensemble Model")
print("Informal → Formal:", generate_sentence_ensemble("तू कहाँ जा रहा है?", direction="informal_to_formal"))
print("Formal → Informal:", generate_sentence_ensemble("आप कहाँ जा रहे हैं?", direction="formal_to_informal"))


Testing Ensemble Model
Informal → Formal: आप कहाँ जा रहे हैं?
Formal → Informal: तुम कहाँ जा रहे हो?


###Ensemble Model inference

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load mT5 model and tokenizer
mt5_tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
mt5_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/mt5_model").to(device)
mt5_model.eval()

# Load IndicBART model and tokenizer
indic_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indicbart", use_fast=False)
indic_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/IndicBART_model").to(device)
indic_model.eval()

# Helper function: Clean output to only keep first sentence
def clean_first_sentence(text):
    # Split on '।' and pick the first part
    first_sentence = text.split('।')[0].strip()
    if first_sentence:
        first_sentence += '।'
    return first_sentence

# Generate output from ensemble (choose longer output, then clean)
def ensemble_generate(prompt_text):
    # --- mT5
    mt5_inputs = mt5_tokenizer(prompt_text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        mt5_output_ids = mt5_model.generate(
            input_ids=mt5_inputs["input_ids"],
            attention_mask=mt5_inputs["attention_mask"],
            max_length=128,
            num_beams=5,
            early_stopping=True,
        )
    mt5_output = mt5_tokenizer.decode(mt5_output_ids[0], skip_special_tokens=True)
    mt5_output = clean_first_sentence(mt5_output)  # 🛠 clean mT5 output

    # --- IndicBART
    indic_inputs = indic_tokenizer(prompt_text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        indic_output_ids = indic_model.generate(
            input_ids=indic_inputs["input_ids"],
            attention_mask=indic_inputs["attention_mask"],
            max_length=128,
            num_beams=5,
            early_stopping=True,
        )
    indic_output = indic_tokenizer.decode(indic_output_ids[0], skip_special_tokens=True)
    indic_output = clean_first_sentence(indic_output)  # 🛠 clean IndicBART output

    # Return longer output (or mT5 default if equal)
    return mt5_output if len(mt5_output) >= len(indic_output) else indic_output

# Batch wrapper for ensemble model
def batch_generate_ensemble(sentences, direction="informal_to_formal"):
    """
    Runs ensemble inference for a batch of sentences in the specified direction.
    """
    outputs = []
    for sentence in sentences:
        prompt = f"{'informal to formal' if direction == 'informal_to_formal' else 'formal to informal'}: {sentence}"
        generated = ensemble_generate(prompt)
        outputs.append(generated)
    return outputs

# Example testing with multiple sentences
informal_sentences = [
    "खाना खा।",
    "बैठ जा।",
    "जल्दी कर।",
    "इस होटल की सर्विस बढ़िया है।",
    "ऑफिस में आ जा, भाई!",
    "फटाफट पेपर दिखा दे।",
    "नई जॉब के लिए एप्लाई कर रहा क्या?",
    "तेरी पढ़ाई तो टॉप लेवल की है।",
    "रिजल्ट चेक कर ले।",
    "शादी में आएगा ना?",
    "टाइम पे काम कर भाई।",
    "यार मीटिंग में टाइम से आ जाना।",
    "तेरी तबियत ठीक चल रही है।",
    "वो मूवी देखी क्या?",
    "तेरी फेवरेट मूवी कौन सी है?",
    "तेरे घरवालों को दिवाली मुबारक!",
    "तेरा स्कूल बहुत अच्छा है।",
    "पेपर क्लियर किया क्या?",
    "तू उससे प्यार करता है क्या?",
    "तेरा प्यार सच्चा लगता है।"
]

formal_sentences = [
    "खाना खाइए।",
    "कृपया बैठ जाइए।",
    "कृपया शीघ्र करें।",
    "भोजन अत्यंत स्वादिष्ट था, धन्यवाद।",
    "क्या आप भोजन का स्वाद पसंद कर रहे हैं?",
    "कृपया अपने दस्तावेज़ प्रस्तुत करें।",
    "आपकी नौकरी में तरक्की हुई है।",
    "क्या आप नई नौकरी के लिए आवेदन कर रहे हैं?",
    "आपकी शिक्षा का स्तर अत्यंत उच्च है।",
    "कृपया परीक्षा परिणाम देखें।",
    "आपका विवाह समारोह बहुत भव्य था।",
    "क्या आप विवाह में सम्मिलित होंगे?",
    "आपके सहयोग के लिए धन्यवाद।",
    "कृपया समय पर कार्य करें।",
    "आपकी मेहनत अत्यंत सराहनीय है।",
    "कृपया मीटिंग में समय पर पहुंचें।",
    "कृपया नियमित रूप से व्यायाम करें।",
    "क्या आपने वह फिल्म देखी है?",
    "आपकी पसंदीदा फिल्म कौन सी है?",
    "त्योहार की हार्दिक शुभकामनाएँ।",
    "आपके परिवार को दीपावली की बधाई।",
    "क्या आप होली खेलेंगे?",
    "कॉलेज की पढ़ाई कठिन है।",
    "क्या आप उससे प्रेम करते हैं?",
    "आपकी दोस्ती सराहनीय है।",
    "आपके प्रेम में सच्चाई झलकती है।"
]

print("=== Ensemble Model (mT5 + IndicBART) output ===")
# Run ensemble test
print("\n🚀 Ensemble Informal → Formal")
formal_outputs = batch_generate_ensemble(informal_sentences, direction="informal_to_formal")
for informal, formal in zip(informal_sentences, formal_outputs):
    print(f"Informal: {informal}")
    print(f"Formal: {formal}\n")

print("\n🚀 Ensemble Formal → Informal")
informal_outputs = batch_generate_ensemble(formal_sentences, direction="formal_to_informal")
for formal, informal in zip(formal_sentences, informal_outputs):
    print(f"Formal: {formal}")
    print(f"Informal: {informal}\n")

=== Ensemble Model (mT5 + IndicBART) output ===

🚀 Ensemble Informal → Formal
Informal: खाना खा।
Formal: कृपया खाना खाइए।

Informal: बैठ जा।
Formal: कृपया बैठ जाइए।

Informal: जल्दी कर।
Formal: कृपया जल्दी करें।

Informal: इस होटल की सर्विस बढ़िया है।
Formal: इस होटल की सर्विस बहुत सराहनीय है।

Informal: ऑफिस में आ जा, भाई!
Formal: कृपया ऑफिस में उपस्थित रहें।

Informal: फटाफट पेपर दिखा दे।
Formal: कृपया मुझे फटाफट पेपर दिखा दीजिए।

Informal: नई जॉब के लिए एप्लाई कर रहा क्या?
Formal: क्या आप नई जॉब के लिए एप्लाई कर रहे हैं?।

Informal: तेरी पढ़ाई तो टॉप लेवल की है।
Formal: आपकी पढ़ाई तो टॉप लेवल की है।

Informal: रिजल्ट चेक कर ले।
Formal: कृपया रिजल्ट चेक करें।

Informal: शादी में आएगा ना?
Formal: क्या आप विवाह समारोह में शामिल होंगे?।

Informal: टाइम पे काम कर भाई।
Formal: कृपया समय पर कार्य करें।

Informal: यार मीटिंग में टाइम से आ जाना।
Formal: कृपया मीटिंग में समय पर उपस्थित रहें।

Informal: तेरी तबियत ठीक चल रही है।
Formal: आपकी तबियत ठीक चल रही है।

Informal: वो मूवी देखी क्या?
F

## Ensemble performance evaluation

In [ ]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import evaluate

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models and tokenizers
mt5_tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
mt5_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/mt5_model").to(device)

indic_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indicbart", use_fast=False)
indic_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Hindi_Formality_Style_Transfer/TST/IndicBART_model").to(device)

# GPT-2 for Perplexity
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Evaluation metrics
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Ensemble Inference Function
def ensemble_generate(text, max_length=128):
    # Prepare inputs
    mt5_input = mt5_tokenizer("informal to formal: " + text, return_tensors="pt", padding=True, truncation=True).to(device)
    indic_input = indic_tokenizer("informal to formal: " + text, return_tensors="pt", padding=True, truncation=True).to(device)

    # Get encoder outputs
    with torch.no_grad():
        mt5_output = mt5_model.generate(
            input_ids=mt5_input["input_ids"],
            attention_mask=mt5_input["attention_mask"],
            max_length=max_length,
            num_beams=5,
            early_stopping=True
        )

        indic_output = indic_model.generate(
            input_ids=indic_input["input_ids"],
            attention_mask=indic_input["attention_mask"],
            max_length=max_length,
            num_beams=5,
            early_stopping=True
        )

    # Decode outputs
    mt5_decoded = mt5_tokenizer.decode(mt5_output[0], skip_special_tokens=True)
    indic_decoded = indic_tokenizer.decode(indic_output[0], skip_special_tokens=True)

    # Simple heuristic: pick the more fluent sentence (lower perplexity)
    mt5_ppl = calculate_perplexity(mt5_decoded)
    indic_ppl = calculate_perplexity(indic_decoded)

    return mt5_decoded if mt5_ppl < indic_ppl else indic_decoded

# Perplexity calculation
def calculate_perplexity(sentence):
    inputs = gpt2_tokenizer(sentence, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        loss = gpt2_model(inputs, labels=inputs).loss
    return torch.exp(loss).item()

# Load test set
test_data = pd.read_csv("/content/drive/MyDrive/TST/test.csv")
# test_data = df[:500]

# Run evaluation
predictions, references, perplexities = [], [], []
correct = 0

for _, row in test_data.iterrows():
    informal = row["informal"]
    formal = row["formal"]

    pred = ensemble_generate(informal)
    ppl = calculate_perplexity(pred)

    predictions.append(pred)
    references.append([formal])
    perplexities.append(ppl)

    if pred.strip() == formal.strip():
        correct += 1

# Compute metrics
accuracy = correct / len(test_data) * 100
bleu_score = bleu.compute(predictions=predictions, references=references)["score"]
rouge_scores = rouge.compute(predictions=predictions, references=references)
berts = bertscore.compute(predictions=predictions, references=[r[0] for r in references], lang="hi")
avg_ppl = sum(perplexities) / len(perplexities)

# Print results
print("\n===== Ensemble Model (mT5 + IndicBART) Evaluation Results =====")
print(f"🔹 Accuracy: {accuracy:.2f}%")
print(f"🔹 BLEU Score: {bleu_score:.2f}")
print(f"🔹 ROUGE: {rouge_scores}")
print(f"🔹 BERTScore Precision: {sum(berts['precision']) / len(berts['precision']):.2f}")
print(f"🔹 BERTScore Recall: {sum(berts['recall']) / len(berts['recall']):.2f}")
print(f"🔹 BERTScore F1: {sum(berts['f1']) / len(berts['f1']):.2f}")
print(f"🔹 Average Perplexity: {avg_ppl:.2f}")


===== Ensemble Model (mT5 + IndicBART) Evaluation Results =====
🔹 Accuracy: 7.51%
🔹 BLEU Score: 21.65
🔹 ROUGE: {'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'rougeLsum': np.float64(0.0)}
🔹 BERTScore Precision: 0.84
🔹 BERTScore Recall: 0.84
🔹 BERTScore F1: 0.84
🔹 Average Perplexity: 6.20
